In [8]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 2.8.0
Default GPU Device: /device:GPU:0


/var/folders/4w/k6c16td51cv9ytcmnnvx3mpw0000gn/T/ipykernel_34904/2432987752.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
2022-04-24 21:57:55.561634: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-24 21:57:55.561680: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-24 21:57:55.561881: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

In [2]:
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

In [3]:
from keras.models import load_model
from keras_retinanet import models

# Load model

In [4]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('keras-retinanet', 'snapshots', sorted(os.listdir('keras-retinanet/snapshots'), reverse=True)[0])

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.convert_model(model)

print(model.summary())

labels_to_names = pd.read_csv('classes.csv', header=None).T.loc[0].to_dict()

Metal device set to: Apple M1


2022-04-24 21:49:47.057296: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-24 21:49:47.057389: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "retinanet-bbox"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1 (Conv2D)                 (None, None, None,   9408        ['input_1[0][0]']                
                                64)                                                               
                                                                                                  
 bn_conv1 (BatchNormalization)  (None, None, None,   256         ['conv1[0][0]']                  
                                64)                                                  

                                                                                                  
 res2b (Add)                    (None, None, None,   0           ['bn2b_branch2c[0][0]',          
                                256)                              'res2a_relu[0][0]']             
                                                                                                  
 res2b_relu (Activation)        (None, None, None,   0           ['res2b[0][0]']                  
                                256)                                                              
                                                                                                  
 res2c_branch2a (Conv2D)        (None, None, None,   16384       ['res2b_relu[0][0]']             
                                64)                                                               
                                                                                                  
 bn2c_bran

 ion)                           128)                                                              
                                                                                                  
 res3b_branch2a_relu (Activatio  (None, None, None,   0          ['bn3b_branch2a[0][0]']          
 n)                             128)                                                              
                                                                                                  
 padding3b_branch2b (ZeroPaddin  (None, None, None,   0          ['res3b_branch2a_relu[0][0]']    
 g2D)                           128)                                                              
                                                                                                  
 res3b_branch2b (Conv2D)        (None, None, None,   147456      ['padding3b_branch2b[0][0]']     
                                128)                                                              
          

 res3d_branch2c (Conv2D)        (None, None, None,   65536       ['res3d_branch2b_relu[0][0]']    
                                512)                                                              
                                                                                                  
 bn3d_branch2c (BatchNormalizat  (None, None, None,   2048       ['res3d_branch2c[0][0]']         
 ion)                           512)                                                              
                                                                                                  
 res3d (Add)                    (None, None, None,   0           ['bn3d_branch2c[0][0]',          
                                512)                              'res3c_relu[0][0]']             
                                                                                                  
 res3d_relu (Activation)        (None, None, None,   0           ['res3d[0][0]']                  
          

                                                                                                  
 res4c_branch2a (Conv2D)        (None, None, None,   262144      ['res4b_relu[0][0]']             
                                256)                                                              
                                                                                                  
 bn4c_branch2a (BatchNormalizat  (None, None, None,   1024       ['res4c_branch2a[0][0]']         
 ion)                           256)                                                              
                                                                                                  
 res4c_branch2a_relu (Activatio  (None, None, None,   0          ['bn4c_branch2a[0][0]']          
 n)                             256)                                                              
                                                                                                  
 padding4c

 ion)                           256)                                                              
                                                                                                  
 res4e_branch2b_relu (Activatio  (None, None, None,   0          ['bn4e_branch2b[0][0]']          
 n)                             256)                                                              
                                                                                                  
 res4e_branch2c (Conv2D)        (None, None, None,   262144      ['res4e_branch2b_relu[0][0]']    
                                1024)                                                             
                                                                                                  
 bn4e_branch2c (BatchNormalizat  (None, None, None,   4096       ['res4e_branch2c[0][0]']         
 ion)                           1024)                                                             
          

 res5a (Add)                    (None, None, None,   0           ['bn5a_branch2c[0][0]',          
                                2048)                             'bn5a_branch1[0][0]']           
                                                                                                  
 res5a_relu (Activation)        (None, None, None,   0           ['res5a[0][0]']                  
                                2048)                                                             
                                                                                                  
 res5b_branch2a (Conv2D)        (None, None, None,   1048576     ['res5a_relu[0][0]']             
                                512)                                                              
                                                                                                  
 bn5b_branch2a (BatchNormalizat  (None, None, None,   2048       ['res5b_branch2a[0][0]']         
 ion)     

                                                                                                  
 P4_upsampled (UpsampleLike)    (None, None, None,   0           ['P4_merged[0][0]',              
                                256)                              'res3d_relu[0][0]']             
                                                                                                  
 C3_reduced (Conv2D)            (None, None, None,   131328      ['res3d_relu[0][0]']             
                                256)                                                              
                                                                                                  
 P6 (Conv2D)                    (None, None, None,   4718848     ['res5c_relu[0][0]']             
                                256)                                                              
                                                                                                  
 P3_merged

# Prediction

In [5]:
import skimage.io as io

def predict(image):
    # Preprocess the image
    image = preprocess_image(image.copy())
    image, scale = resize_image(image)
    # Generate the prediction
    boxes, scores, labels = model.predict_on_batch(
     np.expand_dims(image, axis=0)
     )
    # Scale the output
    boxes /= scale

    return boxes, scores, labels

# Camera connection

In [6]:
camera = cv2.VideoCapture(0)
camera_height = 500

while(True):
    # Take an image (frame)
    _, frame = camera.read()
    frame = cv2.flip(frame, 1)

    # Try prediction
    try:
        boxes, scores, labels = predict(frame)
    except:
        continue
    draw = frame.copy()

    # Draw box
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score > 0.8:
            print(box)
            b = box.astype(int)
            color = label_color(label)
            draw_box(draw, b, color=color)
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)

    # show the frame
    cv2.imshow("Test out", draw)

    key = cv2.waitKey(1)

    # quit camera if 'q' key is pressed
    if key & 0xFF == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()

2022-04-24 21:49:56.130060: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-24 21:49:56.131199: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


[ 792.6575   178.57582 1057.4712   503.81845]
[ 780.14954  177.86122 1045.3348   518.0112 ]
[ 778.5746  181.4557 1043.8662  520.6744]
[ 783.4179   181.71445 1046.9996   506.2599 ]
[ 788.4183   175.31223 1048.0392   492.3405 ]
[ 784.9387   176.19254 1043.498    506.3529 ]
[ 780.4952   174.93167 1045.1775   508.77338]
[ 781.9151   181.45251 1043.0448   503.14432]
[ 781.1815   178.3693  1042.3527   500.72592]
[ 779.07776  171.15239 1037.9469   487.14902]
[ 731.5321   170.72777 1020.72473  556.1416 ]
[248.72275 296.53586 432.63156 617.3209 ]
[820.12445 321.58508 998.8793  566.5518 ]
[806.4397  326.49246 994.99915 576.596  ]
[217.57356 315.48853 410.6234  627.8883 ]
[192.4698  333.64926 393.99097 638.3117 ]
[ 817.20306  333.85757 1001.17755  596.6815 ]
[191.34773 328.44714 389.7306  659.2766 ]
[ 823.1675   335.09833 1005.71173  594.2124 ]
[238.8711  324.4972  503.04932 688.4749 ]
[215.38957 257.93073 473.91385 624.7523 ]
